In [ ]:
import pandas as pd
import numpy as np
import openpyxl
import os
import xlrd
from pathlib import Path
from openpyxl import load_workbook
import re
ARCHIVO_ORIGEN = r"D:\Mateo\ICSI\Proyecto\Data\Raw\Poblacion\Data\Inputs\Censo\censo_pob_sexo_edades_quin_2005.xlsx"
AÑO_CENSO = 2005
df = pd.read_excel(ARCHIVO_ORIGEN)

def procesar_censo_corregido(df):
    """
    Procesa el formato corregido del censo
    """
    datos_procesados = []
    area_actual = {"ubigeo": "", "departamento": ""}
    
    # Reiniciar índices y columnas
    df = df.reset_index(drop=True)
    df.columns = [str(i) for i in range(len(df.columns))]
    
    i = 0
    while i < len(df):
        fila = df.iloc[i]
        
        # 1. Buscar área geográfica (fila con "AREA" en columna 0 y nombre en columna 1)
        if pd.notna(fila['0']) and 'AREA' in str(fila['0']):
            # Extraer ubigeo de la columna 0
            area_texto = str(fila['0'])
            ubigeo_match = re.search(r'(\d{6})', area_texto)
            area_actual['ubigeo'] = ubigeo_match.group(1) if ubigeo_match else ""
            
            # Extraer departamento de la columna 1
            if pd.notna(fila['1']):
                area_actual['departamento'] = str(fila['1']).strip()
            else:
                area_actual['departamento'] = ""
            
            print(f"Encontrada área: Ubigeo={area_actual['ubigeo']}, Departamento={area_actual['departamento']}")
            i += 1
            continue
        
        # 2. Buscar inicio de tabla de datos (fila con "Edad" en columna 0 y "Sexo" en columna 1)
        if (pd.notna(fila['0']) and 'Edad' in str(fila['0']) and 
            pd.notna(fila['1']) and 'Sexo' in str(fila['1'])):
            
            print(f"Encontrado encabezado de tabla en fila {i}")
            
            # La siguiente fila (i+1) tiene los encabezados de sexo
            if i + 1 < len(df):
                fila_encabezados = df.iloc[i + 1]
                
                # Verificar que tiene Hombre, Mujer, Total en las columnas correctas
                if (pd.notna(fila_encabezados['1']) and 'Hombre' in str(fila_encabezados['1']) and
                    pd.notna(fila_encabezados['2']) and 'Mujer' in str(fila_encabezados['2']) and
                    pd.notna(fila_encabezados['3']) and 'Total' in str(fila_encabezados['3'])):
                    
                    print("Encabezados de sexo confirmados")
                    
                    # Procesar filas de datos desde i+2
                    j = i + 2
                    while j < len(df):
                        fila_datos = df.iloc[j]
                        
                        # Si encuentra nueva área, terminar
                        if pd.notna(fila_datos['0']) and 'AREA' in str(fila_datos['0']):
                            break
                            
                        # Si es una fila vacía o de totales, saltar
                        if pd.isna(fila_datos['0']) or str(fila_datos['0']).strip() == '':
                            j += 1
                            continue
                            
                        # Si es fila de datos (contiene "Grupo de" o rangos de edad)
                        if (pd.notna(fila_datos['0']) and 
                            ('Grupo de' in str(fila_datos['0']) or 
                             '-' in str(fila_datos['0']) or 
                             'a' in str(fila_datos['0']))):
                            
                            grupo_edad = str(fila_datos['0']).strip()
                            
                            # Procesar las tres columnas de datos (Hombre, Mujer, Total)
                            for col_idx, sexo in [(1, 'Hombre'), (2, 'Mujer'), (3, 'Total')]:
                                if str(col_idx) in fila_datos.index:
                                    valor = fila_datos[str(col_idx)]
                                    if (pd.notna(valor) and 
                                        str(valor).strip() not in ['', '-', 'nan', 'None']):
                                        try:
                                            # Convertir a número (manejar posibles comas)
                                            valor_str = str(valor).replace(',', '').strip()
                                            valor_num = float(valor_str)
                                            
                                            datos_procesados.append({
                                                'Ubigeo': area_actual['ubigeo'],
                                                'Departamento': area_actual['departamento'],
                                                'grupo_edad': grupo_edad,
                                                'sexo': sexo,
                                                'valor': valor_num
                                            })
                                            
                                        except ValueError:
                                            print(f"Error convirtiendo valor: {valor}")
                                            continue
                        elif 'Total' in str(fila_datos['0']):
                            # Fila de total general, podemos saltarla o procesarla
                            pass
                            
                        j += 1
                    
                    # Avanzar al final del bloque procesado
                    i = j - 1
        
        i += 1
    
    # Crear DataFrame final
    df_final = pd.DataFrame(datos_procesados)
    
    if not df_final.empty:
        # Limpiar grupo_edad
        df_final['grupo_edad'] = df_final['grupo_edad'].str.replace('Grupo de', '').str.strip()
        
        # Convertir valor a numérico
        df_final['valor'] = pd.to_numeric(df_final['valor'], errors='coerce')
        df_final = df_final.dropna(subset=['valor'])
    
    return df_final

# Procesar el DataFrame
df_final = procesar_censo_corregido(df)

print("=== RESULTADO FINAL ===")
print(f"Total de registros: {len(df_final)}")
print(f"\nPrimeros 10 registros:")
print(df_final.head(10))
print(f"\nEstadísticas:")
print(f"Ubigeos únicos: {df_final['Ubigeo'].nunique()}")
print(f"Departamentos: {df_final['Departamento'].unique()}")
print(f"Distribución por sexo:\n{df_final['sexo'].value_counts()}")

def formatear_grupo_edad(grupo_edad):
    if pd.isna(grupo_edad):
        return grupo_edad
    grupo = str(grupo_edad).strip()

    if grupo.startswith('De') and 'años' in grupo:
        return grupo
    
    if '-' in grupo:
        partes = grupo.split('-')
        if len(partes) == 2:
            try:
                inicio = partes[0].strip()
                fin = partes[1].strip()
                return f"De {inicio} a {fin} años"
            except:
                return grupo
    return grupo

#Aplicar la función

df_final['grupo_edad'] = df_final['grupo_edad'].apply(formatear_grupo_edad) 

df_final['Anio'] = 2005 
df_final['Ubigeo'] = df_final['Ubigeo'].astype(str)

import os
ruta_destino = r"D:\Mateo\ICSI\Proyecto\Data\Raw\Poblacion\Data\Processing"
nombre_archivo = "pob_censo_edades_sexo_2005.csv"
ruta_completa = os.path.join(ruta_destino, nombre_archivo)
df_final.to_csv(ruta_completa, index=False, encoding='latin1')


Encontrada área: Ubigeo=010101, Departamento=CHACHAPOYAS
Encontrado encabezado de tabla en fila 3
Encabezados de sexo confirmados
Encontrada área: Ubigeo=010102, Departamento=ASUNCION
Encontrado encabezado de tabla en fila 29
Encabezados de sexo confirmados
Encontrada área: Ubigeo=010103, Departamento=BALSAS
Encontrado encabezado de tabla en fila 54
Encabezados de sexo confirmados
Encontrada área: Ubigeo=010104, Departamento=CHETO
Encontrado encabezado de tabla en fila 80
Encabezados de sexo confirmados
Encontrada área: Ubigeo=010105, Departamento=CHILIQUIN
Encontrado encabezado de tabla en fila 105
Encabezados de sexo confirmados
Encontrada área: Ubigeo=010106, Departamento=CHUQUIBAMBA
Encontrado encabezado de tabla en fila 130
Encabezados de sexo confirmados
Encontrada área: Ubigeo=010107, Departamento=GRANADA
Encontrado encabezado de tabla en fila 156
Encabezados de sexo confirmados
Encontrada área: Ubigeo=010108, Departamento=HUANCAS
Encontrado encabezado de tabla en fila 181
Encab